Fix combinator

lack of polymorphism is a killer here

In [ ]:
from z3 import *

def ListF(T):
    ListF = Datatype("ListF")
    ListF.declare('nil')
    ListF.declare('cons', ('head', T), ('tail', ListF))
    ListF = ListF.create()
    x = Const("x", T)
    f = Const("f", ArraySort(T, T)) # hmm. polymorphism hurts here.
    ListF.map = kd.define_fix("map", [f,x], lambda mp: If(x == ListF.nil, ListF.nil, ListF.cons(f(x.head), mp(f, x.tail))))
    return ListF

def Fix(f):
    Fix = Datatype("Fix")
    T = f(Fix)
    Fix.declare('fix', ('unfix', T))
    Fix = Fix.create()
    
    def induct(P):
        x = FreshConst("x", Fix)
        return  kd.axiom(ForAll([x], Implies(T.map(P,x.unfix), P(x))) == ForAll([x], P(x)))
    
def CoFix(f):
    pass

# Prop Log




In [ ]:
from z3 import *
Prop = Datatype("Prop")
Prop.declare("Atom", ("x", IntSort()))
Prop.declare("Impl", ("a", Prop), ("b", Prop))
Prop.declare("Not", ("x0", Prop))

Prop = Prop.create()

seq = Function("seq", Set(Prop), Prop, BoolSort())
impI = kd.axiom(kd.QForAll([gam1, p,q],   
                seq(Lambda([p, Or(p == p, gam[p])  q),  
        #-----------------------------
                seq(gam, Prop.Impl(p, q)))))
 ))
impE = 

refl = kd.axiom(kd.QForAll(  gam[p], seq(gam, p)))


# Type Metaptheroy

In [1]:
from z3 import *

Type = Datatype('Type')
Type.declare('bool')
Type = Type.create()

Expr = Datatype('Expr')
Expr.declare('true')
Expr.declare('false')
Expr.declare("ite", ('cond', Expr), ('then', Expr), ('else', Expr))
Expr = Expr.create()
e,e1,e2 = Consts('e e1 e2', Expr)

Value = Datatype('Value')
Value.declare("true")
Value = Value.create()

Ctx = Datatype('Ctx')
Ctx.declare('empty')
Ctx.declare("ite", ('cond', Ctx), ('then', Expr), ('else', Expr))
Ctx = Ctx.create()
ctx,ctx1,ctx2 = Consts('ctx ctx1 ctx2', Ctx)



B = BoolSort()
step = Function("step", Ctx, Expr, Ctx, Expr, B)
step_def = ForAll([ctx, e, ctx1, e1],
                  step(ctx, e, ctx1, e1) == 
                    Or()
                  )

eval = TransitiveClosure(step)
# Or labelled transitions + RecFun



https://softwarefoundations.cis.upenn.edu/plf-current/Smallstep.html


In [ ]:
from z3 import *
tm = Datatype("tm")
tm.declare("C", ("n", IntSort()))
tm.declare("P", ("n", tm), ("m", tm))
tm = tm.create()

t, t1, t2 = Consts("t t1 t2", tm)
eval = Function("eval", tm, IntSort())
eval_def = ForAll([x], eval(t) == If(tm.is_C(t), tm.n(t), eval(tm.m(t)) + eval(tm.n(t))))


evalR = Datatype("evalR")
evalR.declare("E_Const", ("tm", tm), ("n", IntSort()))
evalR.declare("E_Plus", ("e1",evalR), ("e2", evalR), ("tm", tm), ("n", IntSort()))
evalR = evalR.create()
e,p,p1,p2 = Consts("e p p1 p2", evalR)

wf = Function("wf", evalR, BoolSort())
wf_def = ForAll([p], wf(p) == Or(
    And(evalR.is_E_Const(p), tm.is_C(evalR.tm(p))), tm.n(evalR.tm(p)) == evalR.n(p),
    And(evalR.is_E_Plus(p), wf(evalR.e1(p)), wf(evalR.e2(p)), tm.is_P(evalR.tm(p)),
        tm.P(evalR.tm(evalR.e1(p)), evalR.tm(evalR.e2(p))) == evalR.tm(p), evalR.n(p) == evalR.n(evalR.e1(p)) + evalR.n(evalR.e2(p)))
    ))
n,n1,n2 = Ints("n1 n2 n")

wf_def2 = ForAll([p], wf(p) == Or(
    Exists([n], evalR(tm.C(x), x) == p),
    Exists([t1,t2,n1,n2,p1,p2]), And(wf(p1), wf(p2), p == evalR.E_Plus(evalR.e1(p), evalR.e2(p), tm.P(evalR.tm(p1), evalR.tm(p2)), evalR.n(p1) + evalR.n(p2)))
    ))


wf2 = Function("wf2", evalR, tm, IntSort(), BoolSort())
wf2_def = ForAll([p, t, n], wf2(p, t,n) == 
                 And(t == evalR.tm(p), n == evalR.n(p), 
                    Or(
                    Exists([n], evalR.E_Const(tm.C(x), x) == p),
                    Exists([t1,t2,n1,n2,p1,p2]), And(wf(p1,t1,n1), wf(p2,t1,n1), 
                                                     p == evalR.E_Plus(evalR.e1(p), evalR.e2(p), tm.P(t1, t2), n1 + n2))
    )))

# So there's another style where I pack the parameters into the  datatype, but also into expose on the well formed function
wf2 = Function("wf2", evalR, tm, IntSort(), BoolSort())
wf2_def = ForAll([p, t, n], wf2(p, t,n) == 
                 And(t == p.tm, n == p.n, 
                    Or(
                    Exists([n], p == evalR.E_Const(tm.C(n), n)),
                    Exists([t1,t2,n1,n2,p1,p2], And(wf(p1,t1,n1), wf(p2,t1,n1), p1 == e.p1, p2 == e.p2,
                                                    p == evalR.E_Plus(p.e1, p.e2, tm.P(t1, t2), n1 + n2))
    )))




Different options  forencodsing inductive rwelationsd

1. Two types. A record type and then is justification type and a well formed predictae that is akin to typeof(p, R(a,b,c))
2. One type. Put the parameters in the justification
3. Make a relation defined recursively Minikaren style



In [ ]:
Even = Datatype("Even")
Even.declare("mk", ("a", IntSort()))
Even.create()

# evidence, term, proof
EvenP = Datatype("EvenP")
EvenP.declare("ev_0")
EvenP.declare("ev_SS", ("p", EvenP))
EvenP = EvenP.create()

RecFunction("typeof", EvenP, Even, BoolSort())
DefineRecFunction(typeof, p, e, 
    Or(And(EvenP.is_ev_0(p), Even.a(e) == 0)
       And(EvenP.is_evSS(p), typeof(EvenP.p(p), Even.mk(Even.a(e) - 2)))
))



Even = Datatype("Even")
Even.declare("ev_0", ("a", IntSort()))
Even.declare("ev_SS", ("a", IntSort()),Eve , wf(Even.p(e)))
    )

)



So there are a bunch of variations of what to make parameters of the well formed predicate vs what to make parameters of the proof object.


I do think this version is the cleanest. Make parameters arguments to the relation and then everything else arguments to the proof objects.
Sometimes might want definefunrec



In [11]:
from z3 import *
even_pf = Datatype("even_pf") # [n]  #params
n = Int("n")
even_pf.declare("even_0")  # pred = lambda n: n == 0
even_pf.declare("even_SS", ("pf", even_pf)) # pred =lambda n, p: even(n-2,p.pf)
pred_ss = lambda n,p: even(n-2, even_pf.pf(p))
even_pf = even_pf.create()
p = Const("p", even_pf)
even = Function("even", IntSort(), even_pf, BoolSort())
even_wf = ForAll([n,p], 
                 even(n,p) == 
                    Or(And(even_pf.is_even_0(p), n == 0), #pf.is_even_0()
                       And(even_pf.is_even_SS(p), pred_ss(n, p))#And(even_pf.is_even_SS(p), even(n-2, even_pf.pf(p))) #pf.is_even_SS()
                    )
                                     )                 

even_wf


ForAll([n, p],
       even(n, p) ==
       Or(And(is(even_0, p), n == 0),
          And(is(even_SS, p), even(n - 2, pf(p)))))

In [18]:

dir(even_pf)
even_pf.is_even_0(p).sexpr()


'((_ is even_0) p)'

In [17]:
foo = Datatype("foo")
type(foo)
foo.declare("bar")
type(foo.create())


    

z3.z3.DatatypeSortRef

# Inductive prototype


In [ ]:
def Inductive(name, f):
    T = Datatype(name)
    fields = f(T)
    for field in fields:
        T.declare(*field)
    return T.create()


In [13]:
from z3 import *
class Inductive(Datatype):
    def __init__(self, name, *args):
        self.args = args
        self.declares = []
        super().__init__(name)
    def declare(self, name, *args, pred=None):
        print(args)
        self.declares.append((name, args, pred))
    def create(self):
        for name,args,pred in self.declares:
            super().declare(name,*args)
        DT = super().create()
        
        wf = Function("wf", DT, BoolSort())
        DT.wf = wf
        
        x = FreshConst(DT, prefix="x")
        ors = []
        for i in range(DT.num_constructors()):
            constructor = DT.constructor(i)
            subwf = [wf(DT.accessor(i,j)(x)) for j in range(constructor.arity()) if constructor.domain(j) == DT]
            ors.append(And(DT.recognizer(i)(x), pred(x), *subwf))
        DT.wf_def = ForAll([x], wf(x) == Or(ors)

        def recursor()

        def induct(P):
            hyps = []
            for i in range(DT.num_constructors()):
                constructor = DT.constructor(i)
                args = [
                    FreshConst(constructor.domain(j), prefix="a")
                    for j in range(constructor.arity())
                ]
                acc = P(constructor(*args))
                for arg in args:
                    if arg.sort() == DT:
                        acc = ForAll([arg], Implies(P(arg), acc))
                    else:
                        acc = ForAll([arg], acc)
                hyps.append(acc)
            x = FreshConst(DT, prefix="x")
            conc = ForAll([x], P(x))
            return Implies(And(hyps), conc)
        DT.induct = induct
        return DT

Nat = Inductive("Nat")
Nat.declare("zero")
Nat.declare("succ", ("pred", Nat))
Nat = Nat.create()

Nat.induct(lambda x: Or(Nat.is_zero(x), Nat.is_succ(x)))
    

()
(('pred', Datatype(Nat, [])),)


Implies(And(Or(is(zero, zero), is(succ, zero)),
            ForAll(a!4,
                   Implies(Or(is(zero, a!4), is(succ, a!4)),
                           Or(is(zero, succ(a!4)),
                              is(succ, succ(a!4)))))),
        ForAll(x!5, Or(is(zero, x!5), is(succ, x!5))))

In [29]:
dir(Nat)
Nat.recognizer(1)(Nat.zero)
Nat.accessor(1,0)
dir(Nat)

['__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'accessor',
 'as_ast',
 'ast',
 'cast',
 'constructor',
 'ctx',
 'ctx_ref',
 'eq',
 'get_id',
 'hash',
 'induct',
 'is_succ',
 'is_zero',
 'kind',
 'name',
 'num_constructors',
 'pred',
 'recognizer',
 'sexpr',
 'subsort',
 'succ',
 'translate',
 'use_pp',
 'zero']

In [ ]:
from knuckledragger.kernel import *
from z3 import *
class Datatype(z3.Datatype):
    def create(self):
        DT = super().create()  # z3 already checks for positivity.
        PredSort = ArraySort(DT, BoolSort())
        # construct an induction principle.
        P = FreshConst(PredSort, prefix="P")
        hyps = []
        for i in range(DT.num_constructors()):
            constructor = DT.constructor(i)
            args = [
                FreshConst(constructor.domain(j), prefix="a")
                for j in range(constructor.arity())
            ]
            acc = P[constructor(*args)]
            for arg in args:
                if arg.sort() == DT:
                    acc = QForAll([arg], P[arg], acc)
                else:
                    acc = ForAll([arg], acc)
            hyps.append(acc)
        x = FreshConst(DT, prefix="x")
        conc = ForAll([x], P[x])
        induct = Lambda([P], Implies(And(hyps), conc))
        induct_ax = trust(ForAll([P], induct[P] == True))

        # recursor

        # store inside the data type?
        DT.induct = induct
        DT.induct_ax = induct_ax
        #DT.rec = rec
        return DT

def recursor(name, *cases, DT):
    assert all(case.range() == DT for case in cases)
    f = z3.RecFunction(name, DT, )  # also freshness needs to be checked


def define_rec(name, args, body, measure=None):
    sig = [arg.sort() for arg in args]
    f = z3.RecFunction(name, *sig)  # also freshness needs to be checked. Yikes.
    # TODO check termination and totality of definition
    RecAddDefinition(f, body)
    fun_def = infer([], ForAll(args, f(*args) == body))
    return f, fun_def

In [9]:
fac = RecFunction('fac', IntSort(), IntSort())
n = Int('n')
RecAddDefinition(fac, n, If(n == 0, 1, n*fac(n-1)))



In [43]:
#def Sigma():
#    pass
# "dependent record" # Structure
def Record(name, *args, pred=None):
    DT = Datatype(name)
    DT.declare("mk_"+name, *args)
    DT = DT.create()
    x = FreshConst(DT, prefix="x")
    if pred != None:
        DT.wf = Function("wf", DT, BoolSort())
        DT.wf_def = ForAll([x], DT.wf(x) == pred(x))
        DT.forall = lambda P: ForAll([x], Implies(pred(x), P(x)))
        DT.exists = lambda P: Exists([x], And(pred(x), P(x)))
    
    #DT.recurse
    #DT.define = lambda name, body: 
    def DTConst(name):
        x = Const(name, DT)
        for i,(name, sort) in enumerate(args):
            setattr(x, name, DT.accessor(0,i)(x))
        return x
    DT.Const = DTConst
    DT.Consts = lambda names: [DTConst(name) for name in names.split()]
    return DT

MyRec = Record("MyRec", ("a", IntSort()), ("b", IntSort()))
q = MyRec.mk_MyRec(1,2)
print(dir(q))
q = MyRec.Const("q")
q.a

# we want to refer to accessors before they exist?
#Even = Record("Even", ("a", IntSort()), ("b", IntSort()), pred=lambda x: Even.a(x) == 2 * Even.b(x))


#z3.DatatypeRef.proj0 = property(lambda self: self.sort().accessor(0,0)(self))
#z3.DatatypeRef.proj1 = property(lambda self: self.sort().accessor(0,1)(self))
z3.DatatypeRef.prj = lambda self, n: self.sort().accessor(0,n)(self)
q = MyRec.mk_MyRec(1,2)
dir(q)

def DTmatch(self, **kwargs):
    pass


q.prj(0)
q.prj(1)

#z3.DatatypeRef.__getattribute__ = lambda self, name: self.sort().__getattribute__(name)(self)
#q.a
DatatypeRef__getattr__ = lambda self, k: self.sort().__getattribute__(k)
#q.sort().__getattribute__("a")
dir(q.sort())

def accessor(self,k):
    sort = self.sort()
    recog = "is_" == k[:3]
    for i in range(sort.num_constructors()):
        cons = sort.constructor(i)
        print(cons.name())
        for j in range(cons.arity()):
            acc = sort.accessor(i,j)
            if acc.name() == k:
                return acc(self)
        if "is_" + cons.name() == k:
            recog = sort.recognizer(i)
            return recog(self)

            
        #constructor = sort.constructor(i)
        #for j in range(constructor.arity()):
        #    if constructor.domain(j) == sort:
        #        yield constructor.accessor(j)

accessor(q,"a")
#dir(q.sort())
accessor(q,"is_mk_MyRec")


['__bool__', '__class__', '__copy__', '__deepcopy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_repr_html_', 'arg', 'as_ast', 'ast', 'children', 'ctx', 'ctx_ref', 'decl', 'eq', 'from_string', 'get_id', 'hash', 'num_args', 'params', 'prj', 'serialize', 'sexpr', 'sort', 'sort_kind', 'translate', 'use_pp']
mk_MyRec
mk_MyRec


is(mk_MyRec, mk_MyRec(1, 2))

In [49]:
foo = Datatype("foo")
foo.declare("bar", ("a", IntSort()))
foo.declare("baz", ("a", IntSort()))
foo = foo.create()
f = Const("f", foo)
prove(foo.accessor(0,0)(f) == foo.accessor(1,0)(f)) # yikes. so this doesn't work.
foo.accessor(0,0)(f) == foo.accessor(1,0)(f)
prove(foo.a(f) == foo.accessor(1,0)(f))
prove(foo.a(f) == foo.accessor(0,0)(f))


counterexample
[f = bar(3), a = [else -> 2]]
proved
counterexample
[f = bar(3), a = [else -> 2]]


In [8]:
def ADT(name, cons):
    T = Datatype(name)
    for c in cons(T):
        if isinstance(c, str):
            T.declare(c)
        else:
            T.declare(*c)
    return T.create()

ADT("Expr", lambda E : [
              "true", 
              "false", 
              ["ite", ("c", E), ("t", E), ("else",E)]])

def Inductive(name, cons):
    T = Datatype(name)
    for c in cons(T):
        if isinstance(c, str):
            T.declare(c)
        else:
            T.declare(*c)
    T.create()
    RecFunction(name, )

class Inductive():
    def __init__(self, name, *args):
        self.name = name
        self.args = args
        self.P = Datatype(name)
        self.recfun = RecFunction(name, P, *[arg.sort() for arg in args], BoolSort())
        self.cons = []
    def declare(self, name, cond *args):
        self.cons.append((name, args))
    def __call__(self, *args):
        return self.recfun(*args)
    def create(self):
        body = 
        RecAddDefinition(self.recfun, self.args, )
        return self.recfun, self.P
        
def Inductive(name, *args):



Expr

https://softwarefoundations.cis.upenn.edu/lf-current/IndProp.html

In [14]:
evenP = Datatype("evenP")
evenP.declare("ev_0")
evenP.declare("ev_SS", ("pred", evenP))
evenP = evenP.create()

def inductP(P):
    return Implies(
        And(P(evenP.ev_0), ForAll([p], Implies(P(p), P(evenP.ev_SS(p))))),
        #-------------------------------------------
        ForAll([p], P(p))
        )



p = Const("p", evenP)
n = Int("n")
even = RecFunction("even", IntSort(), evenP, BoolSort())
RecAddDefinition(even, [n,p], 
    Or(
        And(evenP.is_ev_0(p), n == 0),
        And(evenP.is_ev_SS(p), even(n-2, evenP.pred(p)))
    )
)

def induct(P):
    return inductP(lambda p: ForAll([n], Implies(even(n, p), P(n, p))))

solve(even(6, p))
#prove(Implies(induct(lambda n,p: Not(even(7,p))), Not(even(7,p))))

[p = ev_SS(ev_SS(ev_SS(ev_0))),
 pred = [ev_SS(ev_SS(ev_0)) -> ev_SS(ev_0),
         ev_SS(ev_0) -> ev_0,
         else -> ev_SS(ev_SS(ev_0))],
 even = [else ->
         Or(And(is(ev_0, Var(1)), Var(0) == 0),
            And(is(ev_SS, Var(1)),
                even(Var(0) - 2, pred(Var(1)))))],
 fac = [0 -> 1, else -> Var(0)*fac(Var(0) - 1)]]
failed to prove


Z3Exception: model is not available

In [ ]:


TyDeriv = Datatype("TyDeriv")
TyDeriv.declare("Ttrue")
TyDeriv.declare("Tfalse")
TyDeriv.declare("Tite", ('cond', TyDeriv), ('then', TyDeriv), ('else', TyDeriv))
TyDeriv = TyDeriv.create()

typeof = Function("typeof", Expr, Type, TyDeriv, B)
typeof_def = ForAll([e, t, why],
                    typeof(e, t, why) ==
                    Or(And(why == TyDeriv.Ttrue,  e == Expr.true,  t == Type.bool),
                       And(why == TyDeriv.Tfalse, e == Expr.false, t == Type.bool),
                       And(TyDeriv.is_Tite(why),  e == Expr.ite(e1, e2, e3),
                             
                            t == Type.bool,
                            typeof(e1, Type.bool),
                            typeof(e2, Type.bool),
                            typeof(e3, Type.bool)
                          )
                    )

In [ ]:
TCtx = Datatype('TCtx')
TCtx.declare('nil')
TCtx.declare('cons', ('ctx', TCtx), ('e', Expr), ('v', Value), )
